In [47]:
import json
import os
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

from openAI_utils import checkFT

In [54]:
client = OpenAI(api_key=os.getenv("OAI_KEY"))


In [20]:
def parseJSON(path="../models/mimic-iii/train.json", limit=None):
    f = open(path)
    data = json.load(f)

    limit = len(data) if limit == None else limit

    result = [(e['question'], e['query']) for e in data]
    return result[0:limit]

In [57]:
def toOpenAI(msgPairs, finetune_path="../models/mimic-iii/finetune.jsonl"):
    system_prompt = "You are a computer that provides SQL queries."
    openMessages = [{"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": m[0]}, {"role": "assistant", "content": m[1]}]} for m in msgPairs]

    with open(finetune_path, 'w') as outfile:
        for entry in openMessages:
            json.dump(entry, outfile)
            outfile.write('\n')
    
    return openMessages

In [67]:
def toVec(msgPairs, finetune_path="../models/mimic-iii/vec.txt"):
    openMessages = [m[0] for m in msgPairs]

    with open(finetune_path, 'w') as outfile:
        # for entry in openMessages:
        outfile.write("\n".join([m[0] for m in msgPairs]))
    
    return openMessages

toVec(parseJSON())

['tell me the method of intake of clobetasol propionate 0.05% ointment?',
 'what are the methods of consumption of send 500mg vial?',
 'tell me the intake method for triamcinolone acetonide 0.1% cream?',
 'what is the ingesting method of methimazole?',
 'what are the ways to consume caspofungin?',
 'what is the intake method of the phentolamine mesylate?',
 'tell me the intake method for syringe?',
 'what are the methods of consumption of levetiracetam solution?',
 'tell me the intake method for belatacept study drug (*ind*)?',
 'what are the ways to consume sterile water?',
 'what are the methods of consumption of clotrimazole?',
 'what is the intake method of the albumin 5% (12.5 g)?',
 'what is the ingesting method of nicotine lozenge?',
 'what are the ways to consume dofetilide?',
 'what is the ingesting method of insulin human regular?',
 'tell me the intake method for sirolimus?',
 'what are the ways to consume betamethasone dipro 0.05% oint?',
 'tell me the method of intake of m

In [65]:
finetune_path="../models/mimic-iii/finetune.jsonl"
om = toOpenAI(parseJSON(), finetune_path)

checkFT(om)

No errors found


In [63]:
client.files.create(
  file=open(finetune_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-RwTED2DFZvCzWaPwbAEDRgb0', bytes=112244, created_at=1707614276, filename='finetune.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [64]:
client.fine_tuning.jobs.create(
  training_file="file-RwTED2DFZvCzWaPwbAEDRgb0", 
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}